In [1]:
import random
import os

from imutils.paths import list_images
import tensorflow as tf
from tqdm import tqdm
import numpy as np
import easydict
import cv2

2023-03-22 02:12:17.144956: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
TRAIN_DATASET = ''
TEST_DATASET  = ''

EPOCHS           = 10
IMAGE_SIZE       = (224, 224)
BATCH_SIZE       = 256
BUFFER_SIZE      = BATCH_SIZE * 2
LEARNING_RATE    = 1e-4
STEPS_PER_EPOCH  = 50
VALIDATION_STEPS = 10

AUTO              = tf.data.AUTOTUNE
MODEL_PATH        = os.path.join('output', 'siamese_network')
OUTPUT_IMAGE_PATH = os.path.join('output', 'output_image.png')

In [3]:
class MapFunction:
    
    def __init__(self, image_size):
        self.image_size = image_size
        
    
    def decode_and_resize(self, image_path):
        
        image = tf.io.read_file(image_path)
        image = tf.image.decode_jpeg(image, channels = 3)
        
        image = tf.image.convert_image_dtype(image, dtype = tf.float32)
        image = tf.image.resize(image, self.image_size)
        
        return image
    
    
    def __call__(self, achor, positive, negative):
        
        anchor   = self.decode_and_resize(anchor)
        positive = self.decode_and_resize(positive)
        negative = self.decode_and_resize(negative)
        
        return (anchor, positive, negative)

In [4]:
class TripletGenerator:
    
    def __init__(self, dataset_path):
        
        self.people_names = list()
        for folder_name in os.listdir(dataset_path):
            
            absolute_folder_name = os.path.join(dataset_path, folder_name)
            num_images           = len(os.listdir(absolute_folder_name))
            
            if num_images > 1: self.people_names.append(absolute_folder_name)
            
        self.all_people = self.generate_all_people_dict()
        
        
    def generate_all_people_dict(self):
        
        all_people = dict()
        for person_name in self.people_names:
            
            image_names   = os.listdir(person_name)
            person_photos = [ 
                os.path.join(person_name, image_name) for image_name in image_names
            ]
            all_people[person_name] = person_photos
            
        return all_people
    
    
    def get_next_element(self):
        
        while True:
            
            anchor_name     = random.choice(self.people_names)
            temporary_names = self.people_names.copy()
            temporary_names.remove(anchor_name)
            
            negative_name = random.choice(temporary_names)
            (anchor_photo, positive_photo) = np.random.choice(
                a     = self.all_people[anchor_name],
                size = 2, replace = False
            )
            
            negative_photo = random.choice(self.all_people[negative_name])
            yield (anchor_photo, positive_photo, negative_photo)